# Икс-точка

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('2tEHZEMKjHc')

In [ ]:
from time import sleep
from collections import deque
from copy import deepcopy
from plotly import graph_objects as go
import ipywidgets as widgets
from IPython.display import display

In [ ]:
class Game:                                                                          #konstruktor na igrava
    
    def __init__(self, player_1, player_2):
                                                                                     #player1 i 2 se recnici, so **player_1 se otpakuvaat vrednostite za recnikot prateni so povik na funkcijata
                                                                                     #a dopolnitelno gi dodavame i klucevite simbol i role
        self.player_1 = {**player_1, 'symbol': 'x', 'role': 'MAX'}
        self.player_2 = {**player_2, 'symbol': 'o', 'role': 'MIN'}
        self.next_to_play = self.player_1                                            #prvicno next_to_play e prviot igrac (MAX)
        self.state = [['·', '·', '·'], ['·', '·', '·'], ['·', '·', '·']]             #na pocetok tablata e prazna, a tockata znaci prazno mesto t.e circle open, plotly znae da gi razbere
        self.symbols_fig = {'x': 'x', 'o': 'circle', '·': 'circle-open'}             #recnik kadesto klucot e simbolot a vrednosta e negovoto znacenje
        self.scores = {'x': 1, 'o': -1, 'draw': 0}                                   #pobeda za x e 1, za krugce e -1, nereseno e 0; ova go resava algoritmot za pregled dali imame pobednik ili ne
        self.create_ui()
        hbox = widgets.HBox([self.bt_reset, self.dashboard])
        display(widgets.VBox([hbox, self.fig]))
        self.reset()

    #se zanimava so grafickiot del sledniov del od kodot, ne me interesira
    def reset(self, *args):
        self.next_to_play = self.player_1
        self.update_score('На ред е', self.next_to_play)
        self.state = [['·', '·', '·'], ['·', '·', '·'], ['·', '·', '·']]
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()
        self.evaluated = {}
        self.winner = 'keep_playing'
        self.initiate_turn()

    def create_ui(self):
        self.dashboard = widgets.HTML(description='Статус:', value='')
        self.bt_reset = widgets.Button(description='Ресетирај') #kopceto
        self.bt_reset.on_click(self.reset) #resetirame bezrazlika koj i da bil igracot sto igra 
        self.fig = self.create_fig()

    def create_fig(self): 
                                                                                     #fig e grafikot
                                                                                     #pravi figura, klasa od plotly FW koja ovozmozuva da meniuvame vo nea
                                                                                     #a posle promenite da gi vratime
        N = 3
        fig = go.FigureWidget()
        #vaka kje se raboti so cel da se napravi NxN tabla (ima pisano deka rabotime so 3x3)
        x = [x for y in range(N) for x in range(N)]
        y = [y for y in range(N) for x in range(N)]
        
        #isto taka graficki del
        symbols = [self.symbols_fig[v] for row in self.state for v in row] 
                                                                                    #self.state e momentalnata sostojba na tablata(koj ima izigrano nesto)
                                                                                    #na pocetok site se prazni pa kjezememe circle open (site polinja se prazni) i dopolnitelno nagoduvame debelina na linija i boi...
        fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
                        marker_symbol=symbols, marker_color='LightSkyBlue',
                        marker_line_width=6, marker_line_color='MediumPurple')
                                                                                    #add_scater e dodavanje objekt od tocki ks mode e markers-mnogu tocki
                                                                                    #nivnata pozicija po x i y e od pogorniot red za x i y odnosno 00 01...toest 9 polinja
        
        fig.data[0].on_click(self.human_move)                                       #ako nekoj klikne vrz nekoe kopce se povikuva update fig
        fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, title='x', side='top')     #kazan e razmakot
                                                                                    #lentata x kazuvame da e gore 
        fig.update_yaxes(
            range=[-0.5, N - 0.5], dtick=1, title='y', autorange='reversed')
                                                                                    #lentata y promenuvame da e 0 1 2
        fig.update_layout(width=600, height=600, showlegend=False)                  #kazuvame da e 600 na 600 pikseli bez legenda od strana
        return fig #inicijalizirana e tablatta

    def convert_state_to_symbols(self):
        return [self.symbols_fig[v] for row in self.state for v in row]
     #znaci izminuvame niz sekoj red od state po kolonite (v)
        #na primer razgleduvame 
        #state = [
        #['·', '·', 'x'],
        #['·', 'o', '·'],
        #['x', '·', '·'],
        #]
        #vo nea so izminuvanje na state na primer kje stigneme do red 1, clen 1, toa e 'o'
        #sega v='o'/mora vaka, vo recnik ne se pristapuva preku indeks tuku isklucivo preku kluc 
        #povikuvame self.symbols_fig['o'], so ogled na toa sto e recnik symbols_fig, izlez ke ni dade->circle


 
    def initiate_turn(self):
        if 'human' not in [self.player_1['type'], self.player_2['type']]: #ako igrata ne e nameneta da ja igra covek, tuku ja igraat dve vestacki inteligencii
            while self.winner == 'keep_playing': #se dodeka nema pobednik pomegju niv
                self.ai_move() #se povikuva ai_move
        elif self.next_to_play['type'] == 'AI': #ako barem eden od player1 ili player2 e covek, a ne AI, dokolku sleden da igra e AI
            self.ai_move() #se povikuva ovaa funkcija

    #kako se dvizi AI 
    def ai_move(self):
        self.dashboard.value += ' -- пресметува'
        state = tuple([tuple(row) for row in self.state])               #state e lista od listi (edna lista e edna redica), sega ja konvertirame vo torka od torki
        result, move = self.minimax(state, self.next_to_play['role'])   #povikuva minimax i rezultatot odi vo result (tuka dava best value) i best move
                                                                        #potegot e socinet od x i y i po pauza od 1 sec (ako igraat dve ai da e popregledno)
        sleep(1)
        self.player_took_turn(*move)                                    #potegot se postavuva po 1 sec
                                                                        #so * ja otpakuvame torkata 
                                                                        #i so povik na ovaa funkcija player_took_turn mu davame do znaenje na kompjuterot deka AI go zavrsil svojot del i sega se ceka na sleden cekor
                                                                        #generiran od drug igrac-human, ili AI 

    def human_move(self, trace, points, selector):
        x, y = points.xs[0], points.ys[0]
        keep_playing = self.winner == 'keep_playing'
        human_on_turn = self.next_to_play['type'] == 'human'
        empty_place = self.state[y][x] == '·'
        if keep_playing and human_on_turn and empty_place:
            self.player_took_turn(x, y)
            if self.next_to_play['type'] == 'AI':
                self.ai_move()

    def player_took_turn(self, x, y):
        self.state[y][x] = self.next_to_play['symbol']                   #kazuvame na xy deka stavame simbol soodveten na sledniot igrac x ili tocka 
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols() #i sega iscrtuvame 
        self.winner = self.check_victory(self.state)                     #se povikuva check_victori 
        if self.winner != 'keep_playing':                                #ako ne vratilo keep playing toa bi znacelo 1.ili imame pobednik (x ili tocka, max ili min) 2.nereseno e
            if self.winner == 'draw':
                self.dashboard.value = 'Нерешено.'
            else:
                self.update_score('Победник е', self.next_to_play)       #dosega sme igrale so x na primer i seuste ne mu e smeneta vrednosta, pa znaci otkako x napravil cekor imame pobednik-x e pobednikot
            return
        if self.next_to_play == self.player_2:                           #ako sepak vratilo keep playing, samo si go zamneuvaat redot (ako igral 1, sega na red e 2 i obratno!)
            self.next_to_play = self.player_1
        else:
            self.next_to_play = self.player_2
        self.update_score('На ред е', self.next_to_play) #za segasen 1 pecatime 2

    def update_score(self, message, player):  
        player_data = ' - '.join(list(player.values())[:-1]) 
        self.dashboard.value = f'{message} <b> {player_data} </b>.'

        
        
    #MINIMAXOV RABOTI KAKO PREBARUVANJE PO DLABOCINA-RESENIE SIG KJE NAJDE ZA AI DA NE POBEDI, NO NE OPTIMALNO! (? DALI ZA AI DA NE POBEDI)-PRASAJ GO STEFAN!
    def minimax(self, node, player, alpha=-2, beta=2, depth=0): 
        #tuka se -2 i 2 moze i minus beskraj i plus beskraj
        #no max e 1 za pobeda na x i -1 za pobeda na tockata znaci tuka -2 i 2 imaat efekt na minus plus beskraj
        #koristi alfa beta potkastruvanje
        
        if node not in self.evaluated:                                                      #node e jazelot niz koj kje prebaruvame 
            self.evaluated[node] = self.check_victory(node)
                                                                                            #gledame dali za nea ima pobednik i kje ja smestime vo evaluated
                                                                                            #node e sostojba torka od torki i e pogodna da e kluc za recnik
                                                                                            #ja stavame tuka i nareden pat koga za ista sba treba da prebarame nema vekje da prebaruvame
                                                                                            #tuku direktno kje prodolzime na keep playing
        if self.evaluated[node] != 'keep_playing':                                          #za nova igra se resetira na prazen recnik i dokolku segasnata tabla ne e evaluated
            return self.scores[self.evaluated[node]], None                                  ???TUKA NE RAZBIRAM STO SE SLUCUVA???
        
        #prvo best value e plus beskraj za min odnosno dva i analogno za max
        best_value = 2 if player == 'MIN' else -2
        best_move = None #none bidejkji nekogas kje presmetame rez pa bestmove kje se popolni
        #tuka go imame i potegot za razlika od prvata tetr
        
        #iterirame niz site deca 
        for child, move in self.expand_state(node, player):
            other_player = 'MIN' if player == 'MAX' else 'MAX'
            result, _ = self.minimax(child, other_player, alpha, beta, depth+1) #ja povikuvame minimax za nov jazol (naslednik na node)
            #a other player e min ako sme bile na max ili max ako sme bile na min
            #dlabocinata e 1 ama tuka ne ja koristime
            
            #delov e ist kako vo preth tetratka
            #pri best value odreduvame best move
            if player == 'MIN':
                if result <= alpha:
                    return result, best_move
                if result < beta:
                    beta = result
                if result < best_value:
                    best_value = result
                    best_move = move
            elif player == 'MAX':
                if result >= beta:
                    return result, best_move
                if result > alpha:
                    alpha = result
                if result > best_value:
                    best_value = result
                    best_move = move
        return best_value, best_move

    
    #vrtime niz site redovi na state odnosno element po element od matricata
    #ako ima prazno mesto togas istoto kje se popolni i gi dobivame site mozni situacii 
    #expand_state ni gi dava site mozni situacii i plus gi prakja na minimax za da izbere najdobar od site mozni potezi i najdobra vrednost
    def expand_state(self, state, player):
        symbol = 'x' if player == 'MAX' else 'o'
        for y, row in enumerate(state):                                                     #minuva niz redicite i kolonite
            for x, value in enumerate(row):                      
                if value == '·':                                                            #ako najde prazno mesto 
                    new_state = list([list(row) for row in state])                          #tablata ja pretvorame vo lista od listi za da mozeme vo nea da prezapiseme vrednost na posakuvanata pozicija za pridvizuvanje
                    new_state[y][x] = symbol                                                #na toa prazno mesto od LISTATA postavuvame simbol-ako rabotime so max togas x, ako rabotime so min togas o
                    
                    yield tuple([tuple(row) for row in new_state]), (x, y)                  #i posle promenata vrakjame vo torka od torki povtorno a ne vo lista od listi
                                                                                            #TORKA OD TORKI E POTREBNA VO TAA FORMA ZA MINIMAX
                                                        
    def check_victory(self, state):                                     #proveruva dali imame pobednik
                                                                        #state e nekoja tabla za koja proveruvame dali ima pobednik
    
                                                                        # proveruvame redici, idejata e da se iterira niz site redovi i proveruvame dali prvata
                                                                        #vtorata i tretata vr od toj red se isti 
                                                                        #i ako se isti i pritoa ne se prazno mesto
                                                                        #bi trebalo da ja vratime taa vrednost 
        for y in range(len(state)):
            if state[y][0] == state[y][1] == state[y][2] and state[y][0] != '·':
                return state[y][0]
            
                                                                        # proveruvame po koloni, idejata e kolona po koloa da proveruvame po logika za proverka po redici 
        for x in range(len(state[0])):
            if state[0][x] == state[1][x] == state[2][x] and state[0][x] != '·':
                return state[0][x]
            
                                                                        #proveruvame glavna dijagonala, direktno sporeduvajkji gi indeksite
        if state[0][0] == state[1][1] == state[2][2] and state[0][0] != '·':
            return state[0][0]
        
                                                                        #proveruvame sporedna dijagonala, direktno pristapuvajkji gi indeksite
        if state[0][2] == state[1][1] == state[2][0] and state[0][2] != '·':
            return state[0][2]
        
        #ako stigneme do ovoj moment znaci nemame dosega nisto vrateno pa nema pobednik epa sledno  da vidime dali e nereseno 
        
        #za da e nereseno treba site polinja da se popolneti, i ako se popolneti (a dosega vidovme deka nema pobednik), toa znaci deka e neresenp
        
        if all([value != '·' for row in state for value in row]):
            return 'draw'
                                                                        #ako toa se sluci znaci navistina e nereseno
        
        
        return 'keep_playing'                                           #a ako ne e ispolnet uslovot vo situacija koga nemame najdeno pobednik da ne e voopsto seuste ni popolneta do kraj tablata, toa znaci deka treba
                                                                        #da prodolzime da igrame


game = Game({'name': '1', 'type': 'AI'}, {'name': '2', 'type': 'human'})
#prviot igrac e vestacka inteligencija, vtoriot igrac e covek,igra so tocka
#game=Game(('1','AI','x'),('2','human','o'))

In [3]:
 #Измени во проверката за победник:
def check_victory(state, player):
    for symbol in ['x', 'o']:
        for y in range(3):
            if all(state[y][i] == symbol for i in range(3)) or all(state[i][y] == symbol for i in range(3)):
                return symbol

        if all(state[i][i] == symbol for i in range(3)) or all(state[i][2 - i] == symbol for i in range(3)):
            return symbol


    return 'keep_playing'

## Задачи за самостојна работа

- Симулирајте ја играта [Hexapawn](https://en.wikipedia.org/wiki/Hexapawn) и видете кој е победник.
    - Оваа игра е прва задача на вториот колоквиум 2019.
    - Оваа игра има мало дрво, па можете да ја разграните играта до крај.
    - По можност, направете ја интерактивна, како што е игра икс-точка направена во оваа тетратка.
- Симулирајте ја играта [Connect4](https://en.wikipedia.org/wiki/Connect_Four) и видете кој е победник.
    - Дрвото во оваа игра е преголемо за да го истражите до крај. Затоа, напишете функција која ќе го процени резултатот на играта кога минимакс алгоритамот ќе дојде до однапред одредена длабочина.
    - По можност, направете ја интерактивна, како што е игра икс-точка направена во оваа тетратка.
    - Признак за добро сработена задача: Играта сте ја направиле доволно квалитетно ако самите не можете да ја победите.

### Решенијата во продолжение се од Васе Трендафилов

## Hexapawn

In [ ]:
from plotly import graph_objects as go
import ipywidgets as widgets
from collections import deque
from copy import deepcopy
import time

In [ ]:
def minimaxRoot(depth, board, player):
    global moves
    moves = board.available_moves()
    scores = {}
    bestMove = -9999 if player else 9999

    for move in moves:
        board.push(move)
        value = minimax(depth - 1, board, -10000, 10000, not player)
        scores[move] = value
        board.pop()
        #if value == 1000:
         #   bestMove = value
          #  bestMoveFound = move
           # return bestMoveFound , [],[]
        if player and value >= bestMove:
            bestMove = value
            bestMoveFound = move
        elif not player and value <= bestMove:
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound , moves,scores
def minimax(depth, board, alpha, beta, player):
    if depth == 0:
        return board.score()

    moves = board.available_moves()
   # if not moves:
    #    return 1000

    #за играч има предефинирано да се однесува на MAX. Со помош на not player прави конверзија од MAX во MIN и обратно
    #тука е дадено not player исто како што беше претходно повикана функција other_player
    
    if (player):
        bestMove = -9999
        for move in moves:
            board.push(move)
            bestMove = max(bestMove, minimax(depth - 1, board, alpha, beta, not player))
            board.pop()
            alpha = max(alpha, bestMove)
            if beta <= alpha:
                return bestMove
        return bestMove
    else:
        bestMove = 9999
        for move in moves:
            board.push(move);
            bestMove = min(bestMove, minimax(depth - 1, board, alpha, beta, not player))
            board.pop()
            beta = min(beta, bestMove)
            if beta <= alpha:
                return bestMove
        return bestMove

In [ ]:
class Board:
    def __init__(self, table, turn=True):
        self.table = deepcopy(table)
        self.queue = []
        self.turn = turn 
        
    def push(self,move):
        self.queue.append(move)
        self.update_table()
        self.turn = not self.turn
        
    def pop(self):
        move = self.queue.pop()
        self.update_table(move)
        self.turn = not self.turn
        return move
        
    def peek(self):
        return self.queue[-1]
    
    def update_table(self,move=None):
        if move != None:
            old, new = move
            self.table[old[0]][old[1]] = not self.turn
            self.table[new[0]][new[1]] = None if old[1]==new[1] else self.turn
        else:
            old, new = self.peek()
            self.table[old[0]][old[1]] = None
            self.table[new[0]][new[1]] = self.turn
            
    def legal_moves(self,x,y):
        x += -1 if self.turn else 1
        moves = []
        for i in range(-1,2):
            if 0 <= x < M and 0 <= y+i < N:
                if self.table[x][y+i] == None and i==0:
                    moves.append((x,y))
                elif self.table[x][y+i] == (not self.turn) and i!=0:
                    moves.append((x,y+i))
        return moves
    
    def available_moves(self):
        moves = []
        for x in range(M):
            for y in range(N):
                if self.table[x][y] == self.turn:
                    for move in self.legal_moves(x,y):
                        moves.append(((x,y),move))
        return moves
    
    def score(self):
        score = 0
        for x in range(M):
            for y in range(N):
                if self.table[x][y] != None:
                    score += M-x if self.table[x][y] == True else -(x+1)
                    if x == 0 and self.table[x][y] == True:
                        return 9999
                    elif x == M-1 and self.table[x][y] == False:
                        return -9999
        score += len([i for row in self.table for i in row if i == True]) - len([i for row in self.table for i in row if i == False])
        return score 

In [ ]:
def posible_moves(x,y):
    moves = []
    if TABLE[x][y] == None:
        return moves
    for i in range(-1,2):
        if 0 <= x-1 < M and 0 <= y+i < N:
            if TABLE[x-1][y+i] == None and i==0:
                moves.append((x-1,y))
            elif TABLE[x-1][y+i] == False and i!=0:
                moves.append((x-1,y+i))
    return moves
def game_end(board):
    if False in board.table[M-1]:
        return "Red"
    elif True in board.table[0]:
        return "Blue"
    if not board.available_moves():
        return "Red" if board.turn == True else "Blue"
    return None

In [ ]:
debug = True
moves = None
clicked,xf,yf= 0,0,0
def update_fig(trace, points, selector):
    x,y = points.ys[0],points.xs[0]
    global clicked, xf, yf, moves, TABLE
    clicked +=1  
    
    if clicked == 1:
        xf,yf = x,y
        temp = deepcopy(TABLE)
        moves = posible_moves(x,y)
        if not moves:  clicked = 0
            
        for i,j in moves:
            temp[i][j] = 3 if temp[i][j] == False else 2
            
        colors=['#121212' if n == None 
                else'#ff7f0e' if n == 2 else '#1f77b4' if n == 1 
                else '#d62728' if n == 0 else '#2ca02c' for row in temp for n in row]
        fig.data[0].marker['color'] = colors
        
    elif clicked == 2:
        if (x,y) in moves:
            TABLE[x][y] = True
            TABLE[xf][yf] = None
            
        colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
        fig.data[0].marker['color'] = colors
        clicked,moves = 0,None
        
        if game_end(Board(TABLE,False)) != None:
            text.value = f'<font size=4> {game_end(Board(TABLE,False))} wins !!'
            fig.data[0].on_click(None)
            return
        
        time.sleep(0.5) #AI Part
        board = Board(TABLE,False)
        move,moves,scores = minimaxRoot(4, board, False)
        if debug : 
            print([[move,scores[move]] for move in moves])
            
        board.push(move)
        TABLE = board.table
        colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
        fig.data[0].marker['color'] = colors
        
        if game_end(Board(TABLE)) != None:
            text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
            fig.data[0].on_click(None)
            return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M+1)*10+N+2, 
        marker_symbol='square', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 4
M = 4
TABLE = [
    [False,]*N,
    [None,None,None,None],
    [None,None,None,None],
    [True,]*N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
widgets.VBox([fig,text])

In [ ]:
debug = True
def update_fig(trace, points, selector):
    global TABLE
    
    board = Board(TABLE)
    move,moves,scores = minimaxRoot(4, board, True)
    if debug : 
        print([[move,scores[move]] for move in moves])

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
        fig.data[0].on_click(None)
        return 
    
    time.sleep(1) #AI Part
    board = Board(TABLE,False)
    move,moves,scores = minimaxRoot(5, board, False)
    if debug : 
        print([[move,scores[move]] for move in moves])

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
        fig.data[0].on_click(None)
        return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M+1)*10+N+2, 
        marker_symbol='square', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 6
M = 5
TABLE = [
    [False,]*N,
    [None,]*N,
    [None,]*N,
    [None,]*N,
    [True,]*N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
widgets.VBox([fig,text])

## Connect 4

In [ ]:
from plotly import graph_objects as go
import ipywidgets as widgets
from collections import deque
from copy import deepcopy

In [ ]:
def add_circle(y):
    for i in range(M-1,-1,-1):
        if TABLE[i][y] == None:
            break
    return i, y 
def game_end(board): #vaka proveruva koj e pobednikottt :)))))  FINALLLLLYYYYY
    if abs(board.x_check()) == 1000:
        return "Blue" if board.x_check() == 1000 else "Red"
    if abs(board.y_check()) == 1000:
        return "Blue" if board.y_check() == 1000 else "Red"
    if abs(board.diag_check()) == 1000:
        return "Blue" if board.diag_check() == 1000 else "Red"
    if abs(board.diag_check(False)) == 1000:
        return "Blue" if board.diag_check(False) == 1000 else "Red"
    return None

In [ ]:
def minimaxRoot(depth, board, player):
    global moves
    moves = board.available_moves()
    scores = {}
    bestMove = -9999 if player else 9999

    for move in moves:
        board.push(move)
        value = minimax(depth - 1, board, -10000, 10000, not player)
        scores[move] = value
        board.pop()
        if player and value >= bestMove:
            bestMove = value
            bestMoveFound = move
        elif not player and value <= bestMove:
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound , moves,scores
def minimax(depth, board, alpha, beta, player):
    if depth == 0:
        return board.score()

    moves = board.available_moves()

    if (player):
        bestMove = -9999
        for move in moves:
            board.push(move)
            bestMove = max(bestMove, minimax(depth - 1, board, alpha, beta, not player))
            board.pop()
            alpha = max(alpha, bestMove)
            if beta <= alpha:
                return bestMove
        return bestMove
    else:
        bestMove = 9999
        for move in moves:
            board.push(move);
            bestMove = min(bestMove, minimax(depth - 1, board, alpha, beta, not player))
            board.pop()
            beta = min(beta, bestMove)
            if beta <= alpha:
                return bestMove
        return bestMove

In [ ]:
class Board:
    def __init__(self, table, turn):
        self.table = deepcopy(table)
        self.queue = []
        self.turn = turn 
        
    def push(self,move):
        self.queue.append(move)
        self.update_table()
        self.turn = not self.turn
        
    def pop(self):
        move = self.queue.pop()
        self.update_table(move)
        self.turn = not self.turn
        return move
        
    def peek(self):
        return self.queue[-1]
    
    def update_table(self,move=None):
        if move != None:
            for x in range(M-1,-1,-1):
                if self.table[x][move] == None:
                    break
            if self.table[x][move] != None:
                self.table[x][move] = None
            else:
                self.table[x+1][move] = None
                #zosto nie bi postavile na None, neli i pocetno celata tabla e inicijalizirana so NONE vrednosti
        else:
            y = self.peek()
            for x in range(M-1,-1,-1):
                if self.table[x][y] == None:
                    break
            self.table[x][y] = self.turn
        
    def available_moves(self):
        moves = []
        for y in range(N):
            if self.table[0][y] == None:
                moves.append(y)
        return moves
    
    def score(self):
        score = 0
        score += self.x_check()
        score += self.y_check()
        score += self.diag_check()
        score += self.diag_check(False)
        return score
   

    #dali score vo sekoj del si e pisuvano spored random logika 

    def x_check(self):
        score = 0
        prices = [10,20,40,1000]
        blue, red, blue_start, red_start= -1,-1,-1,-1
        for x in range(M):
            #DA MI SE KAZE STO PRAVI
            #verojatno samo ja kompenzira kaznata
            left = 1+len([y for y in range(N) if self.table[x][y] == None])
            
            for y in range(N):
                #ako vekje imame pobednik
                if red == 3:
                    return -1000
                if blue == 3:
                    return 1000
                
                if self.table[x][y] == True:# blue to red transition KAKO E OVA BLUE TO RED 
                    blue += 1
                    if blue == 0:
                        blue_start = y 
                        #ako stigneme do blue (max) se nagolemuva rangot na blue
                        #ako e prvo najdeno blue togas oznacuvame od kaj pocnuva blue
                        
                    if red >= 0:
                        #no, ako dosega imame najdeno red, mora da mu go resetirame brojacot na -1 (zatoa sto nizata za RED e prekinata)
                        
                        #ova tuka ja dava kaznata odnosno kolku se odzema dokolku redicata na red(min) e prekinata
                        #ako e prekinato samo edno min ne se odzema nisto ili ako red_start==0 zasto toa znaci na pocetok
                        #od redicata imame MIN MAX MAX MAX pa ne ni e bitno deka go napravilo prekinot
                        #no ako e povekje od edno min togas sledi soodvetna kazna 
                        #odnosno namaluvanje na score koga rabotime so MAX (sto ne mu e vo polza)
                        score -= 0 if self.table[x][red_start-1] == True or red_start == 0 else prices[red]/(2*left)
                        red, red_start = -1, -1
                        
                elif self.table[x][y] == False:# red to blue transition
                    red += 1
                    if red == 0:
                        red_start = y 
                    if blue >= 0:
                        score += 0 if self.table[x][blue_start-1] == False or blue_start == 0 else prices[blue]/(2*left)
                        #istoto ko gore
                        #samo tuka se nagolemuva vrednosta na score koga rabotime so MIN (sto ne mu e vo polza)
                        blue, blue_start = -1, -1
                        
                elif self.table[x][y] == None:# to none transition
                    if blue >= 0:
                        score += prices[blue]/(2*left) if self.table[x][blue_start-1] == False or blue_start == 0 else prices[blue]/left        
                    if red >= 0:
                        score -= prices[red]/(2*left) if self.table[x][red_start-1] == True or red_start == 0 else prices[red]/left
                    blue, red, blue_start, red_start= -1,-1,-1,-1
            
            
            #i sega ne sme nasle pobednik da ama mozeme da uvidime koj e vo prednost, i go prajme vaka
            if blue >= 0:# most top case
                score += 0 if self.table[blue_start-1][y] == False else prices[blue]/(2*left) 
                #ako blue e zgolemeno (imame bar edno blue)
                #na score kje dodademe 0 dokolku pred nego ima MIN inaku kje dodademe vrednost prices[blue]/(2*left)
                
            if red >= 0:
                score -= 0 if self.table[red_start-1][y] == True  else prices[red]/(2*left) 
                #isto i tuka 
            blue, red, blue_start, red_start= -1,-1,-1,-1
        return score
    
    def y_check(self):
        score = 0
        prices = [4,9,19,1000]
        blue, red, blue_start, red_start= -1,-1,-1,-1
        for y in range(N):
            left = 1#+len([x for x in range(M) if self.table[x][y] == None])
            for x in range(M-1,-1,-1):
                
                if red == 3:
                    return -1000
                if blue == 3:
                    return 1000
                
                if self.table[x][y] == True:
                    blue += 1
                    if blue == 0:
                        blue_start = x 
                    if red >= 0:
                        red, red_start = -1, -1
                        #vidi tuka ne razbiram zosto gi resetira na -1 i -1 AAAAAAAA epa zatoa sto nizata na red mu e prekinata (PREKINATA E 4KATA STO BI SE FORMIRALAAA!!!!!!!!!!)
                        
                elif self.table[x][y] == False:
                    red += 1
                    if red == 0:
                        red_start = x 
                    if blue >= 0:
                        blue, blue_start = -1, -1
                        
                #MOMENTOT E STO KJE SE IZMINUVA OD KOLONI OD DOLE NAGORE PA AKO VEKJE STIGNEME NA EDNA KOLONA STO E PRAZNA
                #SIGURNI SME DEKA I KOLONITE NAD NEA SE PRAZNI!!! (TOA NE E SLUCAJOT SO REDICITE)
                #NA PRIMER VO REDICA MOZE DA BIDE X X PRAZNO X X X ZATOA STO SE POPOLNUVA SPORED KOLONA SAMATA IGRA
                
                
                #I SEGA VEKJE KOGA SME STIGNATI DO MOMENT DA NAGORE IMAME SAMO PRAZNI POLINJA PRESMETUVAME SCORE
                elif self.table[x][y] == None:
                    if blue >= 0:
                        score += prices[blue]/left  #idejata e kolku pogolema e vrednosta na blue tolku pogolem kje bide score-ot 
                        #i ako bi imale prices[3], togas score=1000 (no ovaa proverka ne bi se izvrsila tuka)
                    if red >= 0:
                        score -= prices[red]/left
                        #isto i tuka
                    blue, red, blue_start, red_start= -1,-1,-1,-1
                    
            blue, red, blue_start, red_start= -1,-1,-1,-1
        return score
       
    def diag_check(self,main=True):#a dali toa znaci deka i gorniot del moze bez blue_start i bez red_start
        score = 0
        blue, red= -1,-1 #ok sega zasega nebitno
        row,col,x,y = 0,0,0,0 #neka gi vaka
        table = self.table[::-1] if main else self.table
        # i sega i za main i za anti diagonal kje pravi ista proverka lol
        
        while row+1!= M or col!= N:
            
            #ako stignes do MAX Pole togas zgolemi go brojacot na blue
            if table[x][y] == True:
                blue += 1
                if red >= 0: red = -1
                #i ako paralelno so toa red pocnalo da broi, znaci deka nizata na red ja prekinavme so ovoj moment 
                #PA GO RESETIRAME BROJACOT ZA RED!
                
            elif table[x][y] == False:
                red += 1
                if blue >= 0:blue = -1
                #istoto e i za blue
                
            elif table[x][y] == None:
                blue, red= -1,-1
                #i sega dokolku poleto e prazno, 
                
            if red == 3:
                return -1000
            if blue == 3:
                return 1000
            
            x-=1 #za da go izminuvame vo obraten redosled 
            y+=1 #isto i ova
            #odam (2,0) pa (1,1) pa (0,2) znaci x se namaluva a y se nagolemuva
            
            
            #prasanje za stefan:
            #kako moze x da e pomalo od 0 a row+1 da e ednakvo na brojot na redici koga sekako brojot na redici bi bil pozitiven broj?
            
            #UTRE DORAZMISLI GO DELO
            if x<0 and row+1 == M:
                col+=1
                x=row
                y=col
                blue, red= -1,-1
            elif x<0 and row+1< M:
                row+=1
                x=row
                y=col
                blue, red= -1,-1
            elif y>N-1:
                col+=1
                y=col
                x=row
                blue, red= -1,-1
        return score
    
    #KOE SCORE DA VRATI KOGA NEMA NIKAKVA PROMENA NA SCORE

In [ ]:
turn = True
debug = True
def update_fig(trace, points, selector): #so povik na update fig se primaat 
    #trace i selctor nebitni, points xs i ys kazuva kaj sme kliknale
    x, y = add_circle(points.xs[0]) #proveruvame dali mestoto e prazno
    global turn
    TABLE[x][y] = turn #znaci moze tuka da izigra covekot a ako e ai nema da se vazi 
    turn = not turn
    
    board = Board(TABLE,False)
    y, moves,scores = minimaxRoot(4, board, False)
    if debug : 
        print([(move,scores[move]) for move in moves])
    x,y = add_circle(y)
    TABLE[x][y] = turn
    turn = not turn
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors
    
    if game_end(Board(TABLE,True)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE,True))} wins !!'
        fig.data[0].on_click(None)


def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
        marker_symbol='circle', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 7
M = 6
TABLE = [
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Connect 4')
widgets.VBox([fig,text])